[구글 코랩(Colab)에서 실행하기](https://colab.research.google.com/github/lovedlim/bigdata_analyst_cert_v2/blob/main/part4/ch8/p8_type2.ipynb)

In [1]:
# 문제정의
# 평가: MAE
# target: TotalCharges
# 최종파일: result.csv(컬럼 1개 pred)

# 라이브러리 및 데이터 불러오기
import pandas as pd

# train = pd.read_csv("churn_train.csv")
# test = pd.read_csv("churn_test.csv")
train = pd.read_csv(
    'https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch8/churn_train.csv'
)
test = pd.read_csv(
    'https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch8/churn_test.csv'
)

# 탐색적 데이터 분석(EDA)
print('===== 데이터 크기 =====')
print('Train Shape:', train.shape)
print('Test Shape:', test.shape)

print('\n ===== 데이터 정보(자료형) =====')
print(train.info())

print('\n ===== train 결측치 수 =====')
print(train.isnull().sum().sum())

print('\n ===== test 결측치 수 =====')
print(test.isnull().sum().sum())

print('\n ===== customerID unique 수 =====')
print(train['customerID'].nunique())

print('\n ===== target 기술 통계 =====')
print(train['TotalCharges'].describe())

===== 데이터 크기 =====
Train Shape: (4116, 19)
Test Shape: (1764, 18)

 ===== 데이터 정보(자료형) =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4116 entries, 0 to 4115
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        4116 non-null   object 
 1   gender            4116 non-null   object 
 2   SeniorCitizen     4116 non-null   int64  
 3   Partner           4116 non-null   object 
 4   Dependents        4116 non-null   object 
 5   tenure            4116 non-null   int64  
 6   PhoneService      4116 non-null   object 
 7   MultipleLines     4116 non-null   object 
 8   InternetService   4116 non-null   object 
 9   OnlineSecurity    4116 non-null   object 
 10  OnlineBackup      4116 non-null   object 
 11  DeviceProtection  4116 non-null   object 
 12  TechSupport       4116 non-null   object 
 13  StreamingTV       4116 non-null   object 
 14  StreamingMovies   4116 non-null   object 
 1

In [2]:
# 데이터 전처리
train = train.drop('customerID', axis=1)
test = test.drop(['customerID'], axis=1)
target = train.pop('TotalCharges')

# 레이블 인코딩
from sklearn.preprocessing import LabelEncoder

cols = train.select_dtypes(include='O').columns

for col in cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])

# 검증데이터 분리
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(
    train, target, test_size=0.2, random_state=0
)

# 랜덤포레스트
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=0)
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)

# MAE
from sklearn.metrics import mean_absolute_error

print(mean_absolute_error(y_val, pred))

# LightGBM
import lightgbm as lgb

lg = lgb.LGBMRegressor(random_state=0, verbose=-1)
lg.fit(X_tr, y_tr)
pred = lg.predict(X_val)
print(mean_absolute_error(y_val, pred))

# 최종 제출 파일
pred = rf.predict(test)
result = pd.DataFrame({'pred': pred})
result.to_csv('result.csv', index=False)

951.0960435538027
952.7925407798712


In [3]:
# 1. pred 행의 수
print(pred.shape)

# 2. 생성한 csv 확인
print(pd.read_csv('result.csv').head())

(1764,)
        pred
0  3707.6212
1   923.7132
2  4057.4078
3   952.6143
4  1322.1638
